In [1]:
import numpy as np
import pandas
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


In [58]:
df = pandas.read_csv('billboard_lyrics_1964-2015.csv')
#print(df.head())
#df.shape
print(df.loc[df['Lyrics'].str.contains('adsbutal')])

ValueError: cannot index with vector containing NA / NaN values

In [59]:
# DATA PREPROCESSING
df_no_whitespace = df.loc[df['Lyrics'].str.isspace() == False]
df_no_whitespace.shape
print(df_no_whitespace.loc[df_no_whitespace['Lyrics'].str.contains('adsbutal')])

      Rank            Song       Artist  Year  \
2933    34  all i wanna do  sheryl crow  1994   
3066    67  all i wanna do  sheryl crow  1995   

                                                 Lyrics  Source  
2933   hit it this aint no disco and it aint no coun...     1.0  
3066   hit it this aint no disco and it aint no coun...     1.0  


In [30]:
cv = CountVectorizer(encoding='latin-1')
data_word_counts = cv.fit_transform(df_no_whitespace['Lyrics'])
data_word_counts.shape

(4866, 42123)

In [45]:
import nltk
import string
import os

from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer

from nltk.corpus import stopwords

stemmer = PorterStemmer()

def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text):
    tokens = nltk.word_tokenize(text)
    tokens_filtered = filter(lambda w: not w in set(stopwords.words('english')), tokens)
    stems = stem_tokens(tokens_filtered, stemmer)
    return stems

result = []

for row in df_no_whitespace.iterrows():
    text = row[1]['Lyrics']
    lowers = text.lower()
    no_punctuation = lowers.translate(None, string.punctuation)
    result.append(no_punctuation)

tfidf = TfidfVectorizer(tokenizer=tokenize, stop_words='english', encoding='latin-1')
tfs = tfidf.fit_transform(result)

In [55]:
print(tfidf.get_feature_names())
tfs.shape
#print(tfs[5,:])

[u'0', u'0120', u'02', u'03', u'04', u'07', u'082', u'1', u'10', u'100', u'1000', u'100000', u'100k', u'1010', u'102', u'104', u'105', u'106', u'109', u'10slook', u'10th', u'11', u'110', u'111123oclock', u'112', u'112we', u'1130', u'12', u'120', u'1205', u'123', u'1230', u'1234', u'125', u'129', u'12move', u'12string', u'12yearold', u'13', u'13th', u'14', u'146', u'14th', u'15', u'15000', u'151', u'15th', u'16', u'16th', u'17', u'1780', u'17th', u'18', u'1800430', u'183', u'185', u'18th', u'19', u'1900mixalot', u'1910', u'1949', u'1951', u'1955', u'1959', u'1963', u'1965', u'196667', u'1968', u'1969', u'1970', u'1972', u'1973', u'1975', u'1979', u'1980', u'1981', u'1982', u'1982and', u'1984', u'1985', u'1989', u'1991', u'1992', u'1994', u'1999', u'1break', u'1choru', u'1i', u'1if', u'1oh', u'1repeat', u'1rollercoast', u'1st', u'1when', u'1you', u'2', u'20', u'200', u'2000', u'2001', u'2002', u'2003', u'2004', u'2004girl', u'2004it', u'2006', u'2007', u'2008', u'2009', u'2010', u'2011',

(4866, 35577)

In [5]:
from sklearn.neural_network import MLPClassifier
from math import ceil
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(400,), random_state=1, verbose=10)

In [7]:
X_train = tfs[:train_index]
y_train = tfs.head(train_index)['Label']
X_test = tfs[train_index:]
y_test = tfs.tail(tfs.shape[0] - train_index)['Label']

In [8]:
clf.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(400,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=10,
       warm_start=False)

In [9]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, clf.predict(X_test)))

0.422110552764
